#  **Fynd-Ai-Intern-Assessment Date 5/12/2025**

#### **Importing Required Librareis**

In [83]:
# import libraries
import os
from dotenv import load_dotenv
import pandas as pd
from openai import OpenAI
import json
import ollama
from ollama import Client
import pandas as pd
import time
import json
import ast
import re
from typing import Any, Optional

#### **Some Data Operations on Data**

In [84]:
# load the dataset
data = pd.read_csv("yelp.csv")

In [85]:
data.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [86]:
data.info() # check for null values and data types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  10000 non-null  object
 1   date         10000 non-null  object
 2   review_id    10000 non-null  object
 3   stars        10000 non-null  int64 
 4   text         10000 non-null  object
 5   type         10000 non-null  object
 6   user_id      10000 non-null  object
 7   cool         10000 non-null  int64 
 8   useful       10000 non-null  int64 
 9   funny        10000 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 781.4+ KB


In [87]:
data.shape # check number of rows and columns
 

(10000, 10)

In [88]:
# extract needed columns
data = data[['text','stars']]

In [89]:
# shap of the new dataframe
data.shape

(10000, 2)

In [90]:
data.head()

,text,stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
2,love the gyro plate. Rice is so good and I als...,4
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5


In [91]:
data.tail() # check last few rows

,text,stars
9995,First visit...Had lunch here today - used my G...,3
9996,Should be called house of deliciousness!\n\nI ...,4
9997,I recently visited Olive and Ivy for business ...,4
9998,My nephew just moved to Scottsdale recently so...,2
9999,4-5 locations.. all 4.5 star average.. I think...,5


In [92]:
# drop null values if any
data = data.dropna()

In [93]:
# print the final shape of the dataset
data.shape

(10000, 2)

In [94]:
# Remove very long reviews (more than 500 words)
data  = data[data['text'].str.split().apply(len)<=500]

In [95]:
data.shape

(9847, 2)

In [96]:
# sampel 200 rows for faster processing
data = data.sample(10, random_state=42).reset_index(drop=True)

In [97]:
# clean the text data
data['text'] = (
    data['text']
    .str.replace('\n', ' ')
    .str.replace('"', "'")
)


In [98]:
print(data)

                                                text  stars
0  This place has a great selection of Korean dis...      4
1  This place is the best place I've found so far...      5
2  Okay, I will start out by saying that I just p...      5
3  My husband and I heard great reviews about thi...      1
4  Where I live in Tempe, I can walk to the end o...      3
5  The place did not disappoint. The moment we st...      5
6  I've been going here for the past 9+ years and...      3
7  I love their turkey and provolone cold sub! De...      5
8  Such a great oriental market!  Unusual fresh p...      4
9  At length: A trip to Kauai for our friends' we...      5


In [99]:
# reset index after dropping rows
data = data.reset_index(drop=True)

In [100]:
# convert text column name to review for better understanding
data = data.rename(columns={'text': 'review'})

In [101]:
# print the column names
print(data.columns)

Index(['review', 'stars'], dtype='object')


In [102]:
review = data['review'].iloc[0] 

# **I am using a dataset size of (10, 2) because the free-tier API key cannot handle processing all 200 records. I also tried running it locally using Ollama, but my system is slow, so I chose to work with only 10 samples**

In [103]:
print("Final shape of the dataset:", data.shape)
print("The columns in the dataset are:", data.columns.tolist())

Final shape of the dataset: (10, 2)
The columns in the dataset are: ['review', 'stars']


#### **LLM Operations On Data**

In [104]:
# load environment variables from .env file
load_dotenv()

True

In [105]:
# get API key from environment variable
api  = os.getenv("GAK")

In [106]:
# set it in Openai client
client = OpenAI(
    api_key=api,
    base_url="https://generativelanguage.googleapis.com/v1beta"
)


In [107]:
# 
prompt_1 = [
    {
        "role": "system",
        "content": """
You classify Yelp reviews into 1-5 stars.
Your output must be valid JSON and must not contain extra text.
Star definitions:
1 = very negative
2 = negative
3 = neutral/mixed
4 = positive
5 = very positive
"""
    },
    {
        "role": "user",
        "content": f"""
Review: "{review}"

Return ONLY the JSON object:
{{
  "predicted_stars": <integer>,
  "explanation": "short reason"
}}
"""
    }
]



In [108]:


response = ollama.chat(
    model="llama3.2:latest",
    messages=prompt_1
)

# Extract only the text from the model
message_text = response["message"]["content"]

print(message_text)

{"predicted_stars": 5, "explanation": "The reviewer uses positive language and mentions specific menu items, as well as a family favorite, suggesting a very positive experience."}


In [109]:
prompt_2 = [
    {
        "role": "system",
        "content": """
You classify Yelp reviews into 1-5 stars.
Your output must be valid JSON and contain no additional text.
Follow the format shown in the examples.
"""
    },
    {
        "role": "user",
        "content": "Review: 'Terrible food, rude staff.'"
    },
    {
        "role": "assistant",
        "content": '{"predicted_stars": 1, "explanation": "very negative experience"}'
    },
    {
        "role": "user",
        "content": "Review: 'Great service and delicious food!'"
    },
    {
        "role": "assistant",
        "content": '{"predicted_stars": 5, "explanation": "very positive"}'
    },
    {
        "role": "user",
        "content": f'Review: "{review}"'
    }
]

In [110]:

response = ollama.chat(
    model="llama3.2:latest",
    messages=prompt_2
)

# Extract only the text from the model
message_text = response["message"]["content"]

print(message_text)

{"predicted_stars": 5, "explanation": ""}


In [111]:
prompt_3 = [
    {
        "role": "system",
        "content": """
Think step-by-step internally but DO NOT reveal your reasoning.
Only return the final JSON output.
Ensure the JSON is valid and contains no extra text.
"""
    },
    {
        "role": "user",
        "content": f"""
Review: "{review}"

Return JSON in the following format:
{{
  "predicted_stars": <integer>,
  "explanation": "brief reason"
}}
"""
    }
]

In [112]:

response = ollama.chat(
    model="llama3.2:latest",
    messages=prompt_3
)

# Extract only the text from the model
message_text = response["message"]["content"]

print(message_text)

{
  "predicted_stars": 4,
  "explanation": "Overall positive review with minor complaints about service speed"


In [113]:
# ---------- CONFIG ----------
MODEL_NAME = "llama3.2:latest"   # change to phi3 or another model if you prefer
OLLAMA_HOST = "http://localhost:11434"
REQUEST_DELAY = 0.2              # seconds between requests (tweak if memory thrashes)
MAX_RETRIES = 3
# ---------------------------

# Connect to local Ollama (reuse your client if already created)
try:
    client = Client(host=OLLAMA_HOST)
except Exception as e:
    raise RuntimeError(f"Failed to create Ollama client: {e}")

In [114]:
# ---------- Helpers ----------
def safe_json_parse(text: str) -> Optional[Any]:
    """Try to parse model output into a Python object.
       Returns parsed object (dict/list) or None."""
    if not isinstance(text, str):
        return None
    txt = text.strip()

    # remove code fences if present
    txt = re.sub(r"^```(?:json)?\s*", "", txt)
    txt = re.sub(r"\s*```$", "", txt)

    # 1) strict JSON
    try:
        return json.loads(txt)
    except Exception:
        pass

    # 2) python literal (single quotes allowed)
    try:
        return ast.literal_eval(txt)
    except Exception:
        pass

    # 3) extract first {...} or [...] block and try again
    m = re.search(r"(\{[\s\S]*?\}|\[[\s\S]*?\])", txt)
    if m:
        candidate = m.group(0)
        try:
            return json.loads(candidate)
        except Exception:
            pass
        try:
            return ast.literal_eval(candidate)
        except Exception:
            pass

    return None

In [115]:

# Extract star number helpers
def extract_star_number_from_parsed(parsed: Any) -> Optional[int]:
    """Given a parsed object (dict/list), try to extract an integer star 1..5."""
    if isinstance(parsed, dict):
        for key in ("predicted_stars", "predicted_rating", "rating", "stars"):
            if key in parsed:
                try:
                    val = int(parsed[key])
                    if 1 <= val <= 5:
                        return val
                except:
                    pass
    return None

In [116]:
def extract_star_number_from_text(text: str) -> Optional[int]:
    """Try to extract integer star 1..5 from text using regex fallback."""
    if not isinstance(text, str):
        return None
    # find standalone digit between 1 and 5 (prefer full word boundaries)
    m = re.search(r"\b([1-5])\b", text)
    if m:
        try:
            return int(m.group(1))
        except:
            pass
    return None

In [117]:

def extract_star_number(text: str) -> Optional[int]:
    """Combine methods: attempt parse then regex fallback."""
    parsed = safe_json_parse(text)
    s = extract_star_number_from_parsed(parsed)
    if s is not None:
        return s
    # fallback to regex on raw text
    return extract_star_number_from_text(text)

In [118]:

def run_1lm(messages, max_retries=MAX_RETRIES, init_backoff=1.0):
    """Call Ollama with retries. Returns raw string response (or '{}' if failure)."""
    backoff = init_backoff
    for attempt in range(1, max_retries + 1):
        try:
            resp = client.chat(model=MODEL_NAME, messages=messages, options={"temperature": 0})
            # expected shape: {'message': {'content': '...'}, ...}
            if isinstance(resp, dict):
                if "message" in resp and isinstance(resp["message"], dict) and "content" in resp["message"]:
                    return resp["message"]["content"]
                if "choices" in resp and len(resp["choices"]) > 0:
                    c = resp["choices"][0]
                    if isinstance(c, dict) and "message" in c and "content" in c["message"]:
                        return c["message"]["content"]
            # fallback: stringify
            return str(resp)
        except Exception as e:
            err = str(e)
            print(f"[run_1lm] attempt {attempt}/{max_retries} error: {err}")
            # Do not aggressively retry on memory / model-not-found errors
            if "memory" in err.lower() or "not found" in err.lower():
                return "{}"
            if attempt < max_retries:
                time.sleep(backoff)
                backoff *= 2
            else:
                return "{}"
    return "{}"

In [119]:


# ---------- Main processing ----------
# Ensure data exists
try:
    _ = data
except NameError:
    raise RuntimeError("DataFrame `data` not found. Load your CSV into `data` before running this cell.")

results = []
n = len(data)
print(f"Starting processing {n} reviews with model {MODEL_NAME} ...")

# Counters for per-prompt JSON validity
p1_json_count = 0
p2_json_count = 0
p3_json_count = 0

# Counters for per-prompt correct predictions
p1_correct = 0
p2_correct = 0
p3_correct = 0

# For per-prompt predicted star arrays (for optional confusion matrix)
p1_preds = []
p2_preds = []
p3_preds = []
actuals = []


Starting processing 10 reviews with model llama3.2:latest ...


In [120]:
# Iterate over each review
for idx, row in data.iterrows():
    review_text = str(row.get("review", "")).strip()
    actual = row.get("stars", None)
    actuals.append(actual)

    # Prompt 1 (zero-shot)
    msgs_1 = [
        {"role": "system", "content": "You classify Yelp reviews into 1-5 stars. Return VALID JSON ONLY. Do not include any text outside the JSON object. Use keys: predicted_stars (integer 1-5), explanation (short string)."},
        {"role": "user", "content": f"Review: \"{review_text}\"\n\nReturn JSON with keys predicted_stars (int 1-5) and explanation (string)."}
    ]
    raw_1 = run_1lm(msgs_1)
    parsed_1 = safe_json_parse(raw_1)
    star_1 = extract_star_number_from_parsed(parsed_1) if parsed_1 is not None else extract_star_number(raw_1)
    if parsed_1 is not None:
        p1_json_count += 1
    p1_preds.append(star_1)
    if star_1 is not None and actual is not None and star_1 == int(actual):
        p1_correct += 1

    # Prompt 2 (few-shot)
    msgs_2 = [
        {"role": "system", "content": "You classify Yelp reviews into 1-5 stars. Return VALID JSON ONLY. Do not include any text outside the JSON object."},
        {"role": "user", "content": "Review: 'Terrible food, rude staff.'"},
        {"role": "assistant", "content": '{"predicted_stars": 1, "explanation": "very negative"}'},
        {"role": "user", "content": "Review: 'Great service and delicious food!'"},
        {"role": "assistant", "content": '{"predicted_stars": 5, "explanation": "very positive"}'},
        {"role": "user", "content": f"Review: \"{review_text}\"\n\nReturn JSON with keys predicted_stars (int 1-5) and explanation (string)."}
    ]
    raw_2 = run_1lm(msgs_2)
    parsed_2 = safe_json_parse(raw_2)
    star_2 = extract_star_number_from_parsed(parsed_2) if parsed_2 is not None else extract_star_number(raw_2)
    if parsed_2 is not None:
        p2_json_count += 1
    p2_preds.append(star_2)
    if star_2 is not None and actual is not None and star_2 == int(actual):
        p2_correct += 1

    # Prompt 3 (chain-of-thought instruction but ask to return only final JSON)
    msgs_3 = [
        {"role": "system", "content": "Think step-by-step internally, but DO NOT reveal your chain-of-thought. Return VALID JSON ONLY with keys predicted_stars (int 1-5) and explanation (string)."},
        {"role": "user", "content": f"Review: \"{review_text}\"\n\nReturn JSON with keys predicted_stars (int 1-5) and explanation (string)."}
    ]
    raw_3 = run_1lm(msgs_3)
    parsed_3 = safe_json_parse(raw_3)
    star_3 = extract_star_number_from_parsed(parsed_3) if parsed_3 is not None else extract_star_number(raw_3)
    if parsed_3 is not None:
        p3_json_count += 1
    p3_preds.append(star_3)
    if star_3 is not None and actual is not None and star_3 == int(actual):
        p3_correct += 1

    # Consolidate numeric predicted star:
    # Prefer parsed JSON numeric field for P1 -> P2 -> P3 (or regex fallback), else None
    predicted = None
    for s in (star_1, star_2, star_3):
        if s is not None:
            predicted = s
            break

    # Build result record
    results.append({
        "review": review_text,
        "actual": actual,
        "p1_raw": raw_1,
        "p1_parsed": parsed_1,
        "p1_star": star_1,
        "p2_raw": raw_2,
        "p2_parsed": parsed_2,
        "p2_star": star_2,
        "p3_raw": raw_3,
        "p3_parsed": parsed_3,
        "p3_star": star_3,
        "predicted_stars": predicted
    })

    # polite delay to avoid thrashing memory / I/O
    time.sleep(REQUEST_DELAY)

    if (idx + 1) % 10 == 0 or (idx + 1) == n:
        print(f"Processed {idx + 1}/{n} reviews. Latest predicted: {predicted}")



Processed 10/10 reviews. Latest predicted: 2


In [121]:
# Save to dataframe + csv
results_df = pd.DataFrame(results)
results_df.to_csv("results_with_predictions.csv", index=False)

# Compute per-prompt metrics
p1_json_rate = p1_json_count / n
p2_json_rate = p2_json_count / n
p3_json_rate = p3_json_count / n

p1_accuracy = p1_correct / n
p2_accuracy = p2_correct / n
p3_accuracy = p3_correct / n

# Overall predicted accuracy (based on consolidated 'predicted_stars' column)
valid_final_mask = results_df["predicted_stars"].notnull()
if valid_final_mask.sum() > 0:
    final_accuracy = (results_df.loc[valid_final_mask, "predicted_stars"].astype(int) == results_df.loc[valid_final_mask, "actual"].astype(int)).mean()
else:
    final_accuracy = 0.0

summary = pd.DataFrame([
    {"prompt": "Prompt 1 (Zero-shot)", "json_validity": p1_json_rate, "accuracy": p1_accuracy},
    {"prompt": "Prompt 2 (Few-shot)",  "json_validity": p2_json_rate, "accuracy": p2_accuracy},
    {"prompt": "Prompt 3 (CoT)",       "json_validity": p3_json_rate, "accuracy": p3_accuracy},
    {"prompt": "Final (first available p1->p2->p3)", "json_validity": valid_final_mask.mean(), "accuracy": final_accuracy}
])

summary.to_csv("evaluation_summary.csv", index=False)

print("Done. Saved results_with_predictions.csv and evaluation_summary.csv")
display(summary)
results_df.head()


Done. Saved results_with_predictions.csv and evaluation_summary.csv


,prompt,json_validity,accuracy
0,Prompt 1 (Zero-shot),0.8,0.4
1,Prompt 2 (Few-shot),1.0,0.7
2,Prompt 3 (CoT),0.5,0.3
3,Final (first available p1->p2->p3),1.0,0.4


,review,actual,p1_raw,p1_parsed,p1_star,p2_raw,p2_parsed,p2_star,p3_raw,p3_parsed,p3_star,predicted_stars
0,This place has a great selection of Korean dis...,4,model='llama3.2:latest' created_at='2025-12-07...,"{'predicted_stars': 4, 'explanation': 'Great s...",4,model='llama3.2:latest' created_at='2025-12-07...,"{'predicted_stars': 5, 'explanation': 'very po...",5,model='llama3.2:latest' created_at='2025-12-07...,None,2,4
1,This place is the best place I've found so far...,5,model='llama3.2:latest' created_at='2025-12-07...,"{'predicted_stars': 4, 'explanation': 'Excelle...",4,model='llama3.2:latest' created_at='2025-12-07...,"{'predicted_stars': 5, 'explanation': 'very po...",5,model='llama3.2:latest' created_at='2025-12-07...,"{'predicted_stars': 4, 'explanation': 'The rev...",4,4
2,"Okay, I will start out by saying that I just p...",5,model='llama3.2:latest' created_at='2025-12-07...,"{'predicted_stars': 4, 'explanation': 'Excelle...",4,model='llama3.2:latest' created_at='2025-12-07...,"{'predicted_stars': 5, 'explanation': 'very po...",5,model='llama3.2:latest' created_at='2025-12-07...,None,2,4
3,My husband and I heard great reviews about thi...,1,model='llama3.2:latest' created_at='2025-12-07...,"{'predicted_stars': 1, 'explanation': 'extreme...",1,model='llama3.2:latest' created_at='2025-12-07...,"{'predicted_stars': 1, 'explanation': 'very ne...",1,model='llama3.2:latest' created_at='2025-12-07...,"{'predicted_stars': 1, 'explanation': 'The rev...",1,1
4,"Where I live in Tempe, I can walk to the end o...",3,model='llama3.2:latest' created_at='2025-12-07...,"{'predicted_stars': 4, 'explanation': 'General...",4,model='llama3.2:latest' created_at='2025-12-07...,"{'predicted_stars': 4, 'explanation': 'mostly ...",4,model='llama3.2:latest' created_at='2025-12-07...,None,2,4
